# Pre-processing

Loading of the graph generator

In [0]:
import os
import subprocess
import time
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DataType, FloatType, MapType
import pyspark.sql.functions as F 

In [0]:
dbutils.widgets.text("username", "", "Please enter your git username")
dbutils.widgets.text("password", "", "Please enter your password")

In [0]:
time.sleep(5)

In [0]:
# username = dbutils.widgets.get("username")
# password = dbutils.widgets.get("password")
username = 'sparkistan'
password = 'Nutella336kJ'
url = "https://" + username + ":" + password + "@github.com/MaximeRedstone/graphsMapReduce.git"
email = username + username + "@gmail.com"
dbutils.widgets.removeAll()

In [0]:
!git config --global user.email $email
!git config --global user.name $username
  
if os.path.exists(os.path.join(os.getcwd(), "graphsMapReduce")):
  !cd graphsMapReduce && git pull
else:
  !git clone $url

!cd graphsMapReduce/generator && make re
!mkdir graphsMapReduce/data

Cloning into 'graphsMapReduce'...
remote: Enumerating objects: 234, done.
remote: Counting objects: 0% (1/234)remote: Counting objects: 1% (3/234)remote: Counting objects: 2% (5/234)remote: Counting objects: 3% (8/234)remote: Counting objects: 4% (10/234)remote: Counting objects: 5% (12/234)remote: Counting objects: 6% (15/234)remote: Counting objects: 7% (17/234)remote: Counting objects: 8% (19/234)remote: Counting objects: 9% (22/234)remote: Counting objects: 10% (24/234)remote: Counting objects: 11% (26/234)remote: Counting objects: 12% (29/234)remote: Counting objects: 13% (31/234)remote: Counting objects: 14% (33/234)remote: Counting objects: 15% (36/234)remote: Counting objects: 16% (38/234)remote: Counting objects: 17% (40/234)remote: Counting objects: 18% (43/234)remote: Counting objects: 19% (45/234)remote: Counting objects: 20% (47/234)remote: Counting objects: 21% (50/234)remote: Counting objects: 22% (52/234)remote: Counting objects: 23% (54/234)remote: Counting objects: 24% (57/234)remote: Counting objects: 25% (59/234)remote: Counting objects: 26% (61/234)remote: Counting objects: 27% (64/234)remote: Counting objects: 28% (66/234)remote: Counting objects: 29% (68/234)remote: Counting objects: 30% (71/234)remote: Counting objects: 31% (73/234)remote: Counting objects: 32% (75/234)remote: Counting objects: 33% (78/234)remote: Counting objects: 34% (80/234)remote: Counting objects: 35% (82/234)remote: Counting objects: 36% (85/234)remote: Counting objects: 37% (87/234)remote: Counting objects: 38% (89/234)remote: Counting objects: 39% (92/234)remote: Counting objects: 40% (94/234)remote: Counting objects: 41% (96/234)remote: Counting objects: 42% (99/234)remote: Counting objects: 43% (101/234)remote: Counting objects: 44% (103/234)remote: Counting objects: 45% (106/234)remote: Counting objects: 46% (108/234)remote: Counting objects: 47% (110/234)remote: Counting objects: 48% (113/234)remote: Counting objects: 49% (115/234)remote: Counting objects: 50% (117/234)remote: Counting objects: 51% (120/234)remote: Counting objects: 52% (122/234)remote: Counting objects: 53% (125/234)remote: Counting objects: 54% (127/234)remote: Counting objects: 55% (129/234)remote: Counting objects: 56% (132/234)remote: Counting objects: 57% (134/234)remote: Counting objects: 58% (136/234)remote: Counting objects: 59% (139/234)remote: Counting objects: 60% (141/234)remote: Counting objects: 61% (143/234)remote: Counting objects: 62% (146/234)remote: Counting objects: 63% (148/234)remote: Counting objects: 64% (150/234)remote: Counting objects: 65% (153/234)remote: Counting objects: 66% (155/234)remote: Counting objects: 67% (157/234)remote: Counting objects: 68% (160/234)remote: Counting objects: 69% (162/234)remote: Counting objects: 70% (164/234)remote: Counting objects: 71% (167/234)remote: Counting objects: 72% (169/234)remote: Counting objects: 73% (171/234)remote: Counting objects: 74% (174/234)remote: Counting objects: 75% (176/234)remote: Counting objects: 76% (178/234)remote: Counting objects: 77% (181/234)remote: Counting objects: 78% (183/234)remote: Counting objects: 79% (185/234)remote: Counting objects: 80% (188/234)remote: Counting objects: 81% (190/234)remote: Counting objects: 82% (192/234)remote: Counting objects: 83% (195/234)remote: Counting objects: 84% (197/234)remote: Counting objects: 85% (199/234)remote: Counting objects: 86% (202/234)remote: Counting objects: 87% (204/234)remote: Counting objects: 88% (206/234)remote: Counting objects: 89% (209/234)remote: Counting objects: 90% (211/234)remote: Counting objects: 91% (213/234)remote: Counting objects: 92% (216/234)remote: Counting objects: 93% (218/234)remote: Counting objects: 94% (220/234)remote: Counting objects: 95% (223/234)remote: Counting objects: 96% (225/234)remote: Counting objects: 97% (227/234)remote: Counting objects: 98% (230/234)remote: Counting obje

In [0]:
user_inputs = {
  "4M": 5,
  "7M": 5
}

filestore_paths_dict = {}
for k, v in user_inputs.items():
  
  filename = "graph_" + k + ".txt"
  rel_filepath = os.path.join("..", "data", filename)
  abs_filepath = os.path.join("graphsMapReduce", "data",filename)
  !cd graphsMapReduce/generator && ./graph_generator $k $rel_filepath $v
  
  filestore_name = "dbfs:/FileStore/" + filename
  dbutils.fs.rm(filestore_name)
  f = open(abs_filepath, "r")
  string = f.read()
  dbutils.fs.put(filestore_name, string)
  filestore_paths_dict[(k,v)] = filestore_name
  
filename = "toy_dataset.txt"
rel_filepath = os.path.join("..", "data", filename)
abs_filepath = os.path.join("graphsMapReduce", "data",filename)
filestore_name = "dbfs:/FileStore/" + filename
f = open(abs_filepath, "r")
string = f.read()
try:
  dbutils.fs.put(filestore_name, string)
except:
  pass
filestore_paths_dict['toy'] = filestore_name

Start program at Tue Mar 16 09:06:51 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 71750
Final number of edges approximation is 358753
End program at Tue Mar 16 09:06:51 2021
Wrote 4194312 bytes.
Start program at Tue Mar 16 09:06:58 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 120449
Final number of edges approximation is 602247
End program at Tue Mar 16 09:06:59 2021
Wrote 7340033 bytes.

# RDD implementation

In [0]:
def processGraphRDD(filepath):
  
  def reduce_ccf(x):
    key = x[0]
    values = x[1]
    min_value = values.pop(values.index(min(values)))
    ret = []
    if min_value < key:
      ret.append((key, min_value))
      for value in values:
        accum.add(1) # là on peu surement add en une fois la taille de la liste
        ret.append((value, min_value)) # là on peut surement zip 
    return (ret)
  
  
  text_file = sc.textFile(filepath)
  
  text_file = text_file.filter(lambda x: "#" not in x)
  text_file_split = text_file.map(lambda x: x.split())
  input = text_file_split.map(lambda x: (int(x[0]), int(x[1])))

  accum = sc.accumulator(1)
  while accum.value > 0:

    accum.value = 0
    print("Start loop ", accum.value)

    # CCF-Iterate
    it_map = input.flatMap(lambda x: ((x[0], x[1]), (x[1], x[0])))
#     it_groupby = it_map.groupByKey().mapValues(lambda x: sorted(x))
    it_groupby = it_map.groupByKey().mapValues(list)
    it_reduce = it_groupby.flatMap(lambda x: reduce_ccf(x))
    
    # CCF-Dedup
    ded_map = it_reduce.map(lambda x: ((x[0], x[1]), None))
    ded_groupby = ded_map.groupByKey().mapValues(list)
    input = ded_groupby.map(lambda x: (x[0][0], x[0][1]))

    viz = input.collect()
    print("End loop ", accum.value)
    print('It groupby: ')
    print(it_groupby.take(15))
    print('It reduce: ')
    print(it_reduce.take(15))
    #print('It input: ')
    
    print('Final ded output: ')
    print(input.take(15))

#     print('IT groupby: ')
#     print(it_groupby.take(15))
#     print('IT reduce: ')
#     print(it_reduce.take(15))
    print('-------------')

  print("Processed file")
#   print(viz)

In [0]:
filestore_name = filestore_paths_dict['toy']
processGraphRDD(filestore_name)

In [0]:
!ls graphsMapReduce/data

In [0]:
event_dir = str(subprocess.check_output(['ls', 'eventlogs']))
event_dir = int(event_dir.split("\'")[1][:-2])
full_old_path = os.path.join('eventlogs', str(event_dir), 'eventlog')
full_new_path = os.path.join('eventlogs', str(event_dir), 'eventlog.json')

In [0]:
!cd graphsMapReduce && git pull

In [0]:
comment = ""
!pip install logzero
!mv $full_old_path $full_new_path
!cd graphsMapReduce/src && python3 main.py -e $event_dir #-c $comment

In [0]:
!cd graphsMapReduce/generator && make fclean
!cd graphsMapReduce && git add -A
!cd graphsMapReduce && git config --global user.email "sparkistan@gmail.com"
!cd graphsMapReduce && git config --global user.name "sparkistan"
!cd graphsMapReduce && git commit -m "Added logs"
!cd graphsMapReduce && git push $url

# DataFrame implementation

In [0]:
def processGraphDF(filepath):

  schema = StructType([
      StructField("key", IntegerType(), True),
      StructField("value", IntegerType(), True)])

  df = spark.read.format('csv').load(filepath, headers=False, delimiter='\t', schema=schema)

  counter = 1
  
  while True:
    counter = 0
    print('Starting new loop')

    # CCF-Iterate

    # Creates the (value, key) pairs and adds them to the dataset
    df = df.na.drop()
    df_inverter = df.select(F.col('value').alias('key'), F.col('key').alias('value'))
    df = df.union(df_inverter)

    # Creates the (key, [values]) pairs
    df = df.groupBy('key').agg(F.collect_list('value').alias('value')).sort('key')

    # Removes the lines verifying key < min(values)
    df = df.withColumn('min', F.array_min('value'))
    df_check = df.where('min<key')
    length_sum = df_check.select(F.size('value')).groupBy().sum().collect()[0][0]

    # The counter is the stop condition, acting as an accumulator: when it is null, the loop will exit
    counter = length_sum - df_check.count()
    print(f'Counter value for this loop: {counter}')
    print('-----------')

    if counter == 0:
      df = df.withColumn('value', F.explode('value'))
      df.collect()
      break

    else:
      df = df_check.withColumn('value', F.explode('value'))


    # Generating the (key, min_value) pairs in df3 and renaming them (key, value)

    df_key_min = df.select('key', F.col('min').alias('value')).dropDuplicates()

    df_value_min = df.select(F.col('value').alias('key'), F.col('min').alias('value'))

    df_value_min = df_value_min.where('key!=value')

    df = df_key_min.union(df_value_min)

    # CCF-Dedup
    df = df.dropDuplicates()
    
#       df.show()


Loop 1
It groupby: 
[(2, [1, 3, 4]), (4, [2, 5]), (6, [7]), (8, [7]), (1, [2]), (3, [2]), (5, [4]), (7, [6, 8])]  
It reduce: 
[(2, 1), (3, 1), (4, 1), (4, 2), (5, 2), (8, 7), (3, 2), (5, 4), (7, 6), (8, 6)]  
Final ded output: 
[(3, 1), (4, 2), (8, 6), (2, 1), (4, 1), (5, 2), (8, 7), (3, 2), (5, 4), (7, 6)]

Loop 2
It groupby: 
[(4, [2, 1, 5]), (2, [4, 1, 5, 3]), (8, [6, 7]), (6, [8, 7]), (3, [1, 2]), (1, [3, 2, 4]), (5, [2, 4]), (7, [8, 6])]  
It reduce: 
[(4, 1), (2, 1), (5, 1), (2, 1), (4, 1), (5, 1), (3, 1), (8, 6), (7, 6), (3, 1), (2, 1), (5, 2), (4, 2), (7, 6), (8, 6)]  
Final ded output: 
[(5, 1), (3, 1), (8, 6), (4, 2), (4, 1), (2, 1), (7, 6), (5, 2)]

In [0]:
filestore_name = filestore_paths_dict[('4M', 5)]
processGraphDF(filestore_name)

Starting new loop
Counter value for this loop: 587752
-----------
Starting new loop
Counter value for this loop: 1163311
-----------
Starting new loop
Counter value for this loop: 2276279
-----------
Starting new loop
Counter value for this loop: 1639836
-----------
Starting new loop
Counter value for this loop: 36684
-----------
Starting new loop

In [0]:
## CLASSICAL VERSION ##

def reduce_ccf(key, values, ac_val):
  #print('Entering CCF: ', ac_val)
  #print(locals()['ac_val'])
  min_value = values.pop(values.index(min(values)))
  ret = {}
  if min_value < key:
    #print('Entering if: ', accum.value)
    accum.add(len(values))
    #print('After adding accum: ', accum.value)
    ret[key] = min_value
    for value in values:
      #print('Entering for: ', accum.value)
      #print(f'Entering loop with value {value} and min value {value}')

       # là on peu surement add en une fois la taille de la liste
      ret[value] = min_value
      #ret.append((value, min_value)) # là on peut surement zip 
  else:
    ret[key] = None
  return ret

reducer = F.udf(lambda x, y: reduce_ccf(x, y, accum.value), MapType(IntegerType(), IntegerType()))

schema = StructType([
    StructField("key", IntegerType(), True),
    StructField("value", IntegerType(), True)])

df = spark.read.format('csv').load(filestore_name, headers=False, delimiter='\t', schema=schema)
accum = sc.accumulator(1)

while accum.value > 0:
  print('Starting new loop', accum.value)
  accum.value = 0
  print('Starting reset loop', accum.value)
  print([x for x, _ in locals().items()])
  # CCF-Iterate
  df = df.na.drop()
  df1 = df.withColumn("key", F.create_map(["key", "value"])).select('key')
  df2 = df.withColumn("key", F.create_map(["value", "key"])).select('key')
  df = df1.union(df2)
  df = df.groupBy(F.map_keys('key')[0].alias('key')).agg(F.collect_list(F.map_values('key')[0]).alias('value')).sort('key')
  df = df.withColumn('reducer', reducer('key', 'value')).select('reducer')

  print(f'Accumulator after group by and with column: {accum.value} ')
  df = df.select(F.explode('reducer'))
  df.collect()

  df = df.na.drop()
  print(f'Accumulator after na drop: {accum.value} ')

  # CCF - Dedup
  df = df.dropDuplicates()
  print('Accumulator value after drop duplicates:', accum.value)

filestore_name = filestore_paths_dict['toy']
processGraphDF(filestore_name)

Starting new loop 1
Starting reset loop 0
['sc', 'sqlContext', 'table', 'sql', 'udf', 'display', 'displayHTML', 'getArgument', 'dbutils', 'spark', '__name__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'os', 'subprocess', 'time', '_i2', 'username', 'password', 'url', 'email', '_i3', '_exit_code', '_i4', 'user_inputs', 'filestore_paths_dict', 'k', 'v', 'filename', 'rel_filepath', 'abs_filepath', 'filestore_name', 'f', 'string', '_i5', '_i6', 'StructType', 'StructField', 'IntegerType', 'StringType', 'DataType', 'FloatType', 'MapType', 'F', 'schema', 'df', 'accum', 'df1', 'df2', '_i7', '_i8', '_i9', 'reduce_ccf', 'reducer']
Accumulator after group by and with column: 0

--------------------------------------------------------------------------- 
 PythonException Traceback (most recent call last)
 <command-3724249596082068> in <module> 
 46 print ( f'Accumulator after group by and with column: {accum.value} ' ) 
 47 df = df . select ( F . explode ( 'reducer' ) ) 
 ---> 48 df . collect ( ) 
 49 
 50 df = df . na . drop ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in collect (self) 
 609 # Default path used in OSS Spark / for non-DF-ACL clusters: 
 610 with SCCallSiteSync ( self . _sc ) as css : 
 --> 611 sock_info = self . _jdf . collectToPython ( ) 
 612 return list ( _load_from_socket ( sock_info , BatchedSerializer ( PickleSerializer ( ) ) ) ) 
 613 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 PythonException : An exception was thrown from a UDF: 'Exception: Accumulator.value cannot be accessed inside tasks', from <command-3724249596082068>, line 24. Full traceback below:
Traceback (most recent call last):
 File "<command-3724249596082068>", line 24, in <lambda>
 File "/databricks/spark/python/pyspark/accumulators.py", line 153, in value
 raise Exception("Accumulator.value cannot be accessed inside tasks")
Exception: Accumulator.value cannot be accessed inside tasks

In [0]:

# from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DataType, FloatType, MapType
# import pyspark.sql.functions as F 
# #import udf, array, concat_ws, collect_list, explode, create_map, map_keys, map_value

# def processGraphDF(filepath):
  
# #   def reduce_ccf(key, values, accum):
# #     min_value = values.pop(values.index(min(values)))
# #     ret = {}
# #     if min_value < key:
# #       ret[key] = min_value
# #       accum.add(len(values))
      
# #       for value in values:
# #         print(f'Entering loop with value {value} and min value {value}')
        
# #          # là on peu surement add en une fois la taille de la liste
# #         ret[value] = min_value
# #         #ret.append((value, min_value)) # là on peut surement zip 
# #     else:
# #       ret[key] = None
    
# #     return ret
  
# #   reducer_schema = StructType([
# #       StructField("map", MapType(IntegerType(), IntegerType()), True),
# #       StructField("counter", IntegerType(), True)])
  
# #  reducer = F.udf(lambda x, y: reduce_ccf(x, y, accum), MapType(IntegerType(), IntegerType()))
  
#   schema = StructType([
#       StructField("key", IntegerType(), True),
#       StructField("value", IntegerType(), True)])

#   df = spark.read.format('csv').load(filestore_name, headers=False, delimiter='\t', schema=schema)

#   accum = sc.accumulator(1)
#   #test_accum = spark.accumulator(1)
# #   breaker = 0
#   while accum.value > 0:
    
  
#     accum.value = 0
#     #accum.reset()
#     print("Start loop ", accum.value)
    
#     # CCF-Iterate
#     df = df.na.drop()
#     df1 = df.withColumn("key", F.create_map(["key", "value"])).select('key')
#     df2 = df.withColumn("key", F.create_map(["value", "key"])).select('key')
#     df = df1.union(df2)
#     print(f'Accumulator after union: {accum.value} ')
# #     print('IT Map result: ')

# #     df.show()
    
#     df = df.groupBy(F.map_keys('key')[0].alias('key')).agg(F.collect_list(F.map_values('key')[0]).alias('value')).sort('key')
# #     print('IT Groupby result: ')
# #     df.show()
#     print(f'Accumulator after group by: {accum.value} ')
#     #df = df.withColumn('length', F.size('value'))
#     df.show()
#     df = df.withColumn('reducer', reducer('key', 'value')).select('reducer')
#     df = df.select(F.explode('reducer'))
    
#     print(f'Accumulator after explode/reducer: {accum.value} ')
#     #df = df.na.drop()
#     print(f'Accumulator after na drop: {accum.value} ')
# #     print('IT Reduce result: ')
# #     df.show()
# #     print('-----------------')

#     # CCF - Dedup
    
#     #print('Duplicates :', df.na.sum())
#     df = df.dropDuplicates()
#     print('Accumulator value before collect:', accum.value)
#     df.collect()
#     df.show()
    
#     print('Accumulator value after drop duplicates:', accum.value)
    
# filestore_name = filestore_paths_dict['toy']
# processGraphDF(filestore_name)